In [ ]:
!pip install transformers datasets


In [ ]:
from datasets import load_dataset

# Load dataset
ds = load_dataset("gretelai/symptom_to_diagnosis")

# Inspect one example
print(ds["train"][0])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.jsonl: 0.00B [00:00, ?B/s]

test.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/853 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/212 [00:00<?, ? examples/s]

{'output_text': 'cervical spondylosis', 'input_text': "I've been having a lot of pain in my neck and back. I've also been having trouble with my balance and coordination. I've been coughing a lot and my limbs feel weak."}


In [ ]:
# Split the test dataset into validation and test sets
ds["validation"] = ds["test"].select(range(100))

In [ ]:
{'input_text': 'fever, cough, sore throat', 'output_text': 'flu'}


{'input_text': 'fever, cough, sore throat', 'output_text': 'flu'}

In [ ]:
# Collect all unique diseases
unique_labels = list(set(ds["train"]["output_text"]))
label2id = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for label, i in label2id.items()}

def convert_to_classification(example):
    return {
        "text": example["input_text"],
        "label": label2id[example["output_text"]]
    }

ds_classification = ds.map(convert_to_classification)


Map:   0%|          | 0/853 [00:00<?, ? examples/s]

Map:   0%|          | 0/212 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
{'text': 'fever, cough, sore throat', 'label': 12}


{'text': 'fever, cough, sore throat', 'label': 12}

In [ ]:
from transformers import AutoTokenizer

model_name = "emilyalsentzer/Bio_ClinicalBERT"  # or "dmis-lab/biobert-base-cased-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True)

encoded = ds_classification.map(tokenize, batched=True)


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/853 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/212 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = len(unique_labels)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)


pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir='./logs',
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded["train"],
    eval_dataset=encoded["test"].select(range(100)), # Use the newly created validation dataset
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

/tmp/ipython-input-326405666.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.639915,0.890000,0.887464
2,No log,0.377908,0.920000,0.921131
3,No log,0.319895,0.900000,0.896020
4,No log,0.166523,0.960000,0.959673
5,0.287000,0.136571,0.970000,0.970241
6,0.287000,0.138602,0.970000,0.970012
7,0.287000,0.112772,0.980000,0.979911
8,0.287000,0.116739,0.980000,0.979911
9,0.287000,0.107969,0.980000,0.979911
10,0.019100,0.108495,0.980000,0.979911


TrainOutput(global_step=1070, training_loss=0.1438711257181435, metrics={'train_runtime': 106.7605, 'train_samples_per_second': 79.898, 'train_steps_per_second': 10.022, 'total_flos': 245068739313300.0, 'train_loss': 0.1438711257181435, 'epoch': 10.0})

In [ ]:
# Evaluate the model on the test dataset
test_results = trainer.evaluate(encoded["test"])

print("Test Results:")
print(test_results)

Test Results:
{'eval_loss': 0.1314130276441574, 'eval_accuracy': 0.9669811320754716, 'eval_f1': 0.9668743167574876, 'eval_runtime': 0.363, 'eval_samples_per_second': 583.948, 'eval_steps_per_second': 74.371, 'epoch': 10.0}


In [ ]:
https://github.com/MLforHealth/MIMIC_Extract.gi

# Task
Fine-tune a language model for sequence classification using the dataset in "extracted_symptoms.jsonl". The dataset is a JSON file containing dictionaries with 'input' and 'output' keys. Split the dataset into training, validation, and test sets, shuffle the data during training, and evaluate the model's performance.

## Load and inspect the data

### Subtask:
Load the JSON dataset from the specified path and inspect its structure to understand the keys for input and output text.


**Reasoning**:
The subtask requires loading a JSON dataset and inspecting its structure. Importing the `json` library is the first step to handle JSON data in Python.



In [3]:
import json


**Reasoning**:
Now that the `json` library is imported, the next step is to define the path to the dataset file, load the data, and inspect its structure as per the instructions.



In [4]:
from google.colab import drive
drive.mount('/content/drive')
file_path = "/content/drive/MyDrive/Datasets/LLMs/MIMIC/extracted_symptoms_new.jsonl"

data = []
with open(file_path, 'r') as f:
    for line in f:
        data.append(json.loads(line))

# Print the first few elements to inspect the structure
for i in range(min(5, len(data))):
    print(data[i])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
{'instruction': "     \n\n     Sex:  M\n\nService:  Medicine\n\nCHIEF COMPLAINT:  Admitted from rehabilitation for\nhypotension (systolic blood pressure to the 70s) and\ndecreased urine output.\n\nHISTORY OF PRESENT ILLNESS:  The patient is a 76-year-old\nmale who had been hospitalized at the  from  through  of \nafter undergoing a left femoral-AT bypass graft and was\nsubsequently discharged to a rehabilitation facility.\n\nOn , he presented again to the  after being found to have a systolic\nblood pressure in the 70s and no urine output for 17 hours.\nA Foley catheter placed at the rehabilitation facility\nyielded 100 cc of murky/brown urine.  There may also have\nbeen purulent discharge at the penile meatus at this time.\n\nOn presentation to the Emergency Department, the patient was\nwithout subjective complaints.  In the Emergency Department,\nhe was fou

**Reasoning**:
The previous attempt to load the data failed because the file path was incorrect. The error message indicates that the path points to a directory, not a file. I need to correct the file path to the actual location of the `extracted_symptoms.jsonl` file within the mounted Google Drive.



In [ ]:
file_path = "/content/drive/MyDrive/Datasets/LLMs/MIMIC/extracted_symptoms_new.jsonl"

data = []
with open(file_path, 'r') as f:
    for line in f:
        data.append(json.loads(line))

# Print the first few elements to inspect the structure
for i in range(min(5, len(data))):
    print(data[i])

{'instruction': "     \n\n     Sex:  M\n\nService:  Medicine\n\nCHIEF COMPLAINT:  Admitted from rehabilitation for\nhypotension (systolic blood pressure to the 70s) and\ndecreased urine output.\n\nHISTORY OF PRESENT ILLNESS:  The patient is a 76-year-old\nmale who had been hospitalized at the  from  through  of \nafter undergoing a left femoral-AT bypass graft and was\nsubsequently discharged to a rehabilitation facility.\n\nOn , he presented again to the  after being found to have a systolic\nblood pressure in the 70s and no urine output for 17 hours.\nA Foley catheter placed at the rehabilitation facility\nyielded 100 cc of murky/brown urine.  There may also have\nbeen purulent discharge at the penile meatus at this time.\n\nOn presentation to the Emergency Department, the patient was\nwithout subjective complaints.  In the Emergency Department,\nhe was found to have systolic blood pressure of 85.  He was\ngiven 6 liters of intravenous fluids and transiently started\non dopamine for 

In [15]:
# Inspect the shape of the tokenized inputs
print("Shape of encoded_train['input_ids'][0]:", encoded_train['input_ids'][0].shape)
print("Shape of encoded_train['attention_mask'][0]:", encoded_train['attention_mask'][0].shape)
print("Shape of encoded_train['label'][0]:", encoded_train['label'][0].shape)


# Inspect the shape of the model's embeddings
print("Shape of model.bert.embeddings.word_embeddings.weight:", model.bert.embeddings.word_embeddings.weight.shape)
print("Shape of model.bert.embeddings.position_embeddings.weight:", model.bert.embeddings.position_embeddings.weight.shape)
print("Shape of model.bert.embeddings.token_type_embeddings.weight:", model.bert.embeddings.token_type_embeddings.weight.shape)

Shape of encoded_train['input_ids'][0]: torch.Size([13])
Shape of encoded_train['attention_mask'][0]: torch.Size([13])
Shape of encoded_train['label'][0]: torch.Size([1572])
Shape of model.bert.embeddings.word_embeddings.weight: torch.Size([28996, 768])
Shape of model.bert.embeddings.position_embeddings.weight: torch.Size([512, 768])
Shape of model.bert.embeddings.token_type_embeddings.weight: torch.Size([2, 768])


## Split the dataset

### Subtask:
Split the loaded dataset into training, validation, and test sets.


**Reasoning**:
Convert the list of dictionaries to a Pandas DataFrame and split it into training, validation, and test sets according to the instructions.



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)

# Split the DataFrame into training and temporary sets
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)

# Split the temporary set into validation and test sets
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Display the shapes of the resulting dataframes
print("Training set shape:", train_df.shape)
print("Validation set shape:", val_df.shape)
print("Test set shape:", test_df.shape)

Training set shape: (35, 3)
Validation set shape: (7, 3)
Test set shape: (8, 3)


## Prepare the data for classification

### Subtask:
Convert the dataset into a format suitable for sequence classification, mapping the output text to numerical labels.


**Reasoning**:
Collect all unique labels from the training, validation, and test sets, create label-to-ID and ID-to-label mappings, and define a function to convert the data into the desired format for sequence classification. Then, apply this function to the training, validation, and test sets.



In [ ]:
# Collect all unique labels from train, val, and test sets
# Assuming 'output' contains a string of labels separated by commas and spaces
all_labels = set()
for dataset in [train_df['output'], val_df['output'], test_df['output']]:
    for labels_str in dataset:
        # Handle potential missing or non-string values
        if isinstance(labels_str, str):
            labels = [label.strip() for label in labels_str.split(',')]
            all_labels.update(labels)

unique_labels = list(all_labels)

# Create label to ID and ID to label mappings
label2id = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for label, i in label2id.items()}

# Define a function to convert to classification format
def convert_to_classification(row):
    # Handle potential missing or non-string values in 'output'
    if isinstance(row["output"], str):
        labels = [label.strip() for label in row["output"].split(',')]
        # Create a multi-hot encoded label vector
        label_vector = [0] * len(unique_labels)
        for label in labels:
            if label in label2id:
                label_vector[label2id[label]] = 1
    else:
        # Handle cases where 'output' is not a string (e.g., NaN)
        label_vector = [0] * len(unique_labels) # Or handle as appropriate for your data

    return {
        "text": row["input"], # Use 'input' key here
        "label": label_vector
    }

# Apply the function to create new dataframes
train_classification = train_df.apply(convert_to_classification, axis=1).to_list()
val_classification = val_df.apply(convert_to_classification, axis=1).to_list()
test_classification = test_df.apply(convert_to_classification, axis=1).to_list()

# Display the first example from the training classification data
print(train_classification[0])

{'text': 'Present Symptoms:', 'label': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


## Define metrics and training arguments

### Subtask:
Define the evaluation metrics and training arguments for the model.

**Reasoning**:
Define the evaluation metrics, such as accuracy and F1 score, and set up the training arguments, including the output directory, evaluation strategy, learning rate, batch size, number of epochs, and weight decay.

In [ ]:
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from datasets import Dataset # Import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import json
from google.colab import drive
import torch # Import torch

# Mount Google Drive and define file path (assuming it's the same as before)
drive.mount('/content/drive')
file_path = "/content/drive/MyDrive/Datasets/LLMs/MIMIC/extracted_symptoms.jsonl"

# Load the data
data = []
with open(file_path, 'r') as f:
    for line in f:
        data.append(json.loads(line))

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)

# Split the DataFrame into training and temporary sets
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)

# Split the temporary set into validation and test sets
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)


# Define model_name and tokenizer
model_name = "emilyalsentzer/Bio_ClinicalBERT"  # or "dmis-lab/biobert-base-cased-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Collect all unique labels from train, val, and test sets
# Assuming 'output' contains a string of labels separated by commas and spaces
all_labels = set()
for dataset in [train_df['output'], val_df['output'], test_df['output']]:
    for labels_str in dataset:
        # Handle potential missing or non-string values
        if isinstance(labels_str, str):
            labels = [label.strip() for label in labels_str.split(',')]
            all_labels.update(labels)

unique_labels = list(all_labels)

# Create label to ID and ID to label mappings
label2id = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for label, i in label2id.items()}

# Load the pre-trained model
num_labels = len(unique_labels)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)

# Define a function to convert to classification format
def convert_to_classification(row):
    # Handle potential missing or non-string values in 'output'
    if isinstance(row["output"], str):
        labels = [label.strip() for label in row["output"].split(',')]
        # Create a multi-hot encoded label vector
        label_vector = [0] * len(unique_labels)
        for label in labels:
            if label in label2id:
                label_vector[label2id[label]] = 1
    else:
        # Handle cases where 'output' is not a string (e.g., NaN)
        label_vector = [0] * len(unique_labels) # Or handle as appropriate for your data

    return {
        "text": row["input"], # Use 'input' key here
        "label": label_vector
    }

# Apply the function to create new dataframes
train_classification = train_df.apply(convert_to_classification, axis=1).to_list()
val_classification = val_df.apply(convert_to_classification, axis=1).to_list()

# Convert the list of dictionaries to a Dataset object
train_dataset = Dataset.from_list(train_classification)
val_dataset = Dataset.from_list(val_classification)


def tokenize(batch):
    # Tokenize the text and return the input_ids and attention_mask
    tokenized_inputs = tokenizer(batch["text"], padding="max_length", truncation=True)
    # Include the labels in the returned dictionary
    tokenized_inputs["label"] = batch["label"]
    return tokenized_inputs


# Tokenize the datasets - the 'label' column is already present from convert_to_classification
encoded_train = train_dataset.map(tokenize, batched=True)
encoded_val = val_dataset.map(tokenize, batched=True)

# Set the format of the datasets to 'torch'
encoded_train.set_format("torch", columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
encoded_val.set_format("torch", columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions # Get the raw logits

    # Convert labels to a 2D numpy array of floats
    true_labels_binary = np.array(labels).astype(float)

    # Apply sigmoid to logits to get probabilities
    sigmoid = lambda x: 1 / (1 + np.exp(-x))
    probabilities = sigmoid(preds)

    # Choose a threshold. A common starting point is 0.5
    threshold = 0.5
    predicted_labels_binary = (probabilities > threshold).astype(int)


    # Calculate metrics. For multi-label, 'weighted' F1 averages the F1 score
    # for each label, weighted by the number of true instances for each label.
    # Accuracy in multi-label can be interpreted in different ways (e.g., instance-based, label-based).
    # Here, we'll calculate label-based accuracy and weighted F1.

    # Label-based accuracy: proportion of labels correctly predicted across all instances.
    # This is the proportion of instances where ALL labels were correctly predicted.
    label_accuracy = np.mean((predicted_labels_binary == true_labels_binary).all(axis=1))

    # Weighted F1-score:
    weighted_f1 = f1_score(true_labels_binary, predicted_labels_binary, average="weighted")

    # AUC-ROC score: Can be calculated per label and averaged, or micro/macro averaged.
    # Let's calculate the weighted AUC-ROC
    try:
        weighted_roc_auc = roc_auc_score(true_labels_binary, probabilities, average="weighted")
    except ValueError:
        # Handle cases where there's only one class present in a fold
        weighted_roc_auc = np.nan # Or some other indicator

    return {
        "label_accuracy": label_accuracy,
        "weighted_f1": weighted_f1,
        "weighted_roc_auc": weighted_roc_auc,
    }


training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4, # Reduced batch size
    per_device_eval_batch_size=4,  # Reduced batch size
    num_train_epochs=5, # Reduced number of epochs
    weight_decay=0.01,
    logging_dir='./logs',
    report_to="none",
    # Add parameters for multi-label classification if needed by the Trainer,
    # but generally, the loss function in AutoModelForSequenceClassification
    # handles multi-label when num_labels > 1 and a sigmoid is applied internally
    # before the loss calculation (e.g., BCEWithLogitsLoss).
    # Ensure your model output is logits before the sigmoid for the loss.
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train, # Labels are already in the dataset
    eval_dataset=encoded_val,   # Labels are already in the dataset
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/35 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/7 [00:00<?, ? examples/s]

/tmp/ipython-input-325779138.py:166: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


## Load the pre-trained model

### Subtask:
Load a pre-trained language model for sequence classification.

**Reasoning**:
Load a pre-trained model for sequence classification using the specified model name and the created label mappings.

In [ ]:
from transformers import AutoModelForSequenceClassification

# Define model_name as used in the tokenization step
model_name = "emilyalsentzer/Bio_ClinicalBERT"  # or "dmis-lab/biobert-base-cased-v1.1"
num_labels = len(unique_labels)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Tokenize the data

### Subtask:
Tokenize the input text using a pre-trained tokenizer.


**Reasoning**:
Tokenize the input text using the specified pre-trained tokenizer and apply padding and truncation.



# Task
Create a dataset from "extracted_symptoms_new.jsonl" where each item is a dictionary with 'input' and 'output' keys, and train a Bio_medical Bert model for sequence classification using this dataset.

## Load and inspect the data

### Subtask:
Load the JSON dataset from the specified path and inspect its structure to understand the keys for input and output text.


## Tokenize the data

### Subtask:
Tokenize the input text using a pre-trained tokenizer.


**Reasoning**:
Define the tokenize function and apply it to the datasets, then set the format to torch.



In [5]:
def tokenize(batch):
    tokenized_inputs = tokenizer(batch["text"], padding="max_length", truncation=True)
    tokenized_inputs["label"] = batch["label"]
    return tokenized_inputs

encoded_train = train_dataset.map(tokenize, batched=True)
encoded_val = val_dataset.map(tokenize, batched=True)

encoded_train.set_format("torch", columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
encoded_val.set_format("torch", columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

print(encoded_train[0])
print(encoded_val[0])

NameError: name 'train_dataset' is not defined

**Reasoning**:
The previous code failed because `train_dataset` and `val_dataset` were not defined in the current execution. I need to regenerate the code block that defines and creates these datasets from the dataframes.



In [6]:
from datasets import Dataset # Import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import json
from google.colab import drive
import torch # Import torch

# Mount Google Drive and define file path (assuming it's the same as before)
drive.mount('/content/drive')
file_path = "/content/drive/MyDrive/Datasets/LLMs/MIMIC/extracted_symptoms.jsonl"

# Load the data
data = []
with open(file_path, 'r') as f:
    for line in f:
        data.append(json.loads(line))

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)

# Split the DataFrame into training and temporary sets
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)

# Split the temporary set into validation and test sets
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)


# Define model_name and tokenizer (assuming tokenizer is already loaded)
# model_name = "emilyalsentzer/Bio_ClinicalBERT"  # or "dmis-lab/biobert-base-cased-v1.1"
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# Collect all unique labels from train, val, and test sets
# Assuming 'output' contains a string of labels separated by commas and spaces
all_labels = set()
for dataset in [train_df['output'], val_df['output'], test_df['output']]:
    for labels_str in dataset:
        # Handle potential missing or non-string values
        if isinstance(labels_str, str):
            labels = [label.strip() for label in labels_str.split(',')]
            all_labels.update(labels)

unique_labels = list(all_labels)

# Create label to ID and ID to label mappings
label2id = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for label, i in label2id.items()}


# Define a function to convert to classification format
def convert_to_classification(row):
    # Handle potential missing or non-string values in 'output'
    if isinstance(row["output"], str):
        labels = [label.strip() for label in row["output"].split(',')]
        # Create a multi-hot encoded label vector
        label_vector = [0] * len(unique_labels)
        for label in labels:
            if label in label2id:
                label_vector[label2id[label]] = 1
    else:
        # Handle cases where 'output' is not a string (e.g., NaN)
        label_vector = [0] * len(unique_labels) # Or handle as appropriate for your data

    return {
        "text": row["input"], # Use 'input' key here
        "label": label_vector
    }

# Apply the function to create new dataframes
train_classification = train_df.apply(convert_to_classification, axis=1).to_list()
val_classification = val_df.apply(convert_to_classification, axis=1).to_list()

# Convert the list of dictionaries to a Dataset object
train_dataset = Dataset.from_list(train_classification)
val_dataset = Dataset.from_list(val_classification)


def tokenize(batch):
    # Tokenize the text and return the input_ids and attention_mask
    tokenized_inputs = tokenizer(batch["text"], padding="max_length", truncation=True)
    # Include the labels in the returned dictionary
    tokenized_inputs["label"] = batch["label"]
    return tokenized_inputs


# Tokenize the datasets - the 'label' column is already present from convert_to_classification
encoded_train = train_dataset.map(tokenize, batched=True)
encoded_val = val_dataset.map(tokenize, batched=True)

# Set the format of the datasets to 'torch'
encoded_train.set_format("torch", columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
encoded_val.set_format("torch", columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

print(encoded_train[0])
print(encoded_val[0])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Map:   0%|          | 0/35 [00:00<?, ? examples/s]

NameError: name 'tokenizer' is not defined

**Reasoning**:
The previous code failed because the `tokenizer` was not defined. I need to import and initialize the tokenizer before using it in the `tokenize` function.



In [7]:
from transformers import AutoTokenizer

# Define model_name and tokenizer
model_name = "emilyalsentzer/Bio_ClinicalBERT"  # or "dmis-lab/biobert-base-cased-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    # Tokenize the text and return the input_ids and attention_mask
    tokenized_inputs = tokenizer(batch["text"], padding="max_length", truncation=True)
    # Include the labels in the returned dictionary
    tokenized_inputs["label"] = batch["label"]
    return tokenized_inputs


# Tokenize the datasets - the 'label' column is already present from convert_to_classification
encoded_train = train_dataset.map(tokenize, batched=True)
encoded_val = val_dataset.map(tokenize, batched=True)

# Set the format of the datasets to 'torch'
encoded_train.set_format("torch", columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
encoded_val.set_format("torch", columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

print(encoded_train[0])
print(encoded_val[0])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/35 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/7 [00:00<?, ? examples/s]

{'label': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'input_ids': tensor([ 101, 1675, 8006,  131,  102]), 'token_type_ids': tensor([0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1

## Train the model

### Subtask:
Train the sequence classification model using the prepared dataset and defined arguments.


**Reasoning**:
Train the model using the trainer object and the prepared datasets.



In [8]:
trainer.train()

NameError: name 'trainer' is not defined

**Reasoning**:
The `trainer` object was not defined in the current execution environment. I need to re-execute the cell that defines the `trainer` object, which also includes defining the metrics and training arguments, loading the model, and tokenizing the data.



In [9]:
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from datasets import Dataset # Import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import json
from google.colab import drive
import torch # Import torch

# Mount Google Drive and define file path (assuming it's the same as before)
drive.mount('/content/drive')
file_path = "/content/drive/MyDrive/Datasets/LLMs/MIMIC/extracted_symptoms_new.jsonl"

# Load the data
data = []
with open(file_path, 'r') as f:
    for line in f:
        data.append(json.loads(line))

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)

# Split the DataFrame into training and temporary sets
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)

# Split the temporary set into validation and test sets
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)


# Define model_name and tokenizer
model_name = "emilyalsentzer/Bio_ClinicalBERT"  # or "dmis-lab/biobert-base-cased-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Collect all unique labels from train, val, and test sets
# Assuming 'output' contains a string of labels separated by commas and spaces
all_labels = set()
for dataset in [train_df['output'], val_df['output'], test_df['output']]:
    for labels_str in dataset:
        # Handle potential missing or non-string values
        if isinstance(labels_str, str):
            labels = [label.strip() for label in labels_str.split(',')]
            all_labels.update(labels)

unique_labels = list(all_labels)

# Create label to ID and ID to label mappings
label2id = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for label, i in label2id.items()}

# Load the pre-trained model
num_labels = len(unique_labels)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)

# Define a function to convert to classification format
def convert_to_classification(row):
    # Handle potential missing or non-string values in 'output'
    if isinstance(row["output"], str):
        labels = [label.strip() for label in row["output"].split(',')]
        # Create a multi-hot encoded label vector
        label_vector = [0] * len(unique_labels)
        for label in labels:
            if label in label2id:
                label_vector[label2id[label]] = 1
    else:
        # Handle cases where 'output' is not a string (e.g., NaN)
        label_vector = [0] * len(unique_labels) # Or handle as appropriate for your data

    return {
        "text": row["input"], # Use 'input' key here
        "label": label_vector
    }

# Apply the function to create new dataframes
train_classification = train_df.apply(convert_to_classification, axis=1).to_list()
val_classification = val_df.apply(convert_to_classification, axis=1).to_list()

# Convert the list of dictionaries to a Dataset object
train_dataset = Dataset.from_list(train_classification)
val_dataset = Dataset.from_list(val_classification)


def tokenize(batch):
    # Tokenize the text and return the input_ids and attention_mask
    tokenized_inputs = tokenizer(batch["text"], padding="max_length", truncation=True)
    # Include the labels in the returned dictionary
    tokenized_inputs["label"] = batch["label"]
    return tokenized_inputs


# Tokenize the datasets - the 'label' column is already present from convert_to_classification
encoded_train = train_dataset.map(tokenize, batched=True)
encoded_val = val_dataset.map(tokenize, batched=True)

# Set the format of the datasets to 'torch'
encoded_train.set_format("torch", columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
encoded_val.set_format("torch", columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions # Get the raw logits

    # Convert labels to a 2D numpy array of floats
    true_labels_binary = np.array(labels).astype(float)

    # Apply sigmoid to logits to get probabilities
    sigmoid = lambda x: 1 / (1 + np.exp(-x))
    probabilities = sigmoid(preds)

    # Choose a threshold. A common starting point is 0.5
    threshold = 0.5
    predicted_labels_binary = (probabilities > threshold).astype(int)


    # Calculate metrics. For multi-label, 'weighted' F1 averages the F1 score
    # for each label, weighted by the number of true instances for each label.
    # Accuracy in multi-label can be interpreted in different ways (e.g., instance-based, label-based).
    # Here, we'll calculate label-based accuracy and weighted F1.

    # Label-based accuracy: proportion of instances where ALL labels were correctly predicted.
    label_accuracy = np.mean((predicted_labels_binary == true_labels_binary).all(axis=1))

    # Weighted F1-score:
    weighted_f1 = f1_score(true_labels_binary, predicted_labels_binary, average="weighted")

    # AUC-ROC score: Can be calculated per label and averaged, or micro/macro averaged.
    # Let's calculate the weighted AUC-ROC
    try:
        weighted_roc_auc = roc_auc_score(true_labels_binary, probabilities, average="weighted")
    except ValueError:
        # Handle cases where there's only one class present in a fold
        weighted_roc_auc = np.nan # Or some other indicator


    return {
        "label_accuracy": label_accuracy,
        "weighted_f1": weighted_f1,
        "weighted_roc_auc": weighted_roc_auc,
    }


training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4, # Reduced batch size
    per_device_eval_batch_size=4,  # Reduced batch size
    num_train_epochs=5, # Reduced number of epochs
    weight_decay=0.01,
    logging_dir='./logs',
    report_to="none",
    # Add parameters for multi-label classification if needed by the Trainer,
    # but generally, the loss function in AutoModelForSequenceClassification
    # handles multi-label when num_labels > 1 and a sigmoid is applied internally
    # before the loss calculation (e.g., BCEWithLogitsLoss).
    # Ensure your model output is logits before the sigmoid for the loss.
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train, # Labels are already in the dataset
    eval_dataset=encoded_val,   # Labels are already in the dataset
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/700 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/150 [00:00<?, ? examples/s]

/tmp/ipython-input-4042469629.py:166: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

ValueError: Expected input batch_size (4) to match target batch_size (6288).

**Reasoning**:
The error message "ValueError: Expected input batch_size (4) to match target batch_size (6288)" indicates an issue with the shape of the labels being passed to the loss function during training. This is likely because the model is configured for single-label classification (CrossEntropyLoss), but the labels are in a multi-hot encoded format suitable for multi-label classification (BCEWithLogitsLoss). I need to explicitly set the `problem_type` in the model configuration to "multi_label_classification" to ensure the correct loss function is used. I will re-run the cell that loads the model and defines the trainer with this modification.



In [10]:
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from datasets import Dataset # Import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import json
from google.colab import drive
import torch # Import torch

# Mount Google Drive and define file path (assuming it's the same as before)
drive.mount('/content/drive')
file_path = "/content/drive/MyDrive/Datasets/LLMs/MIMIC/extracted_symptoms_new.jsonl"

# Load the data
data = []
with open(file_path, 'r') as f:
    for line in f:
        data.append(json.loads(line))

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)

# Split the DataFrame into training and temporary sets
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)

# Split the temporary set into validation and test sets
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)


# Define model_name and tokenizer
model_name = "emilyalsentzer/Bio_ClinicalBERT"  # or "dmis-lab/biobert-base-cased-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Collect all unique labels from train, val, and test sets
# Assuming 'output' contains a string of labels separated by commas and spaces
all_labels = set()
for dataset in [train_df['output'], val_df['output'], test_df['output']]:
    for labels_str in dataset:
        # Handle potential missing or non-string values
        if isinstance(labels_str, str):
            labels = [label.strip() for label in labels_str.split(',')]
            all_labels.update(labels)

unique_labels = list(all_labels)

# Create label to ID and ID to label mappings
label2id = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for label, i in label2id.items()}

# Load the pre-trained model
num_labels = len(unique_labels)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    problem_type="multi_label_classification" # Set problem type for multi-label
)

# Define a function to convert to classification format
def convert_to_classification(row):
    # Handle potential missing or non-string values in 'output'
    if isinstance(row["output"], str):
        labels = [label.strip() for label in row["output"].split(',')]
        # Create a multi-hot encoded label vector
        label_vector = [0] * len(unique_labels)
        for label in labels:
            if label in label2id:
                label_vector[label2id[label]] = 1
    else:
        # Handle cases where 'output' is not a string (e.g., NaN)
        label_vector = [0] * len(unique_labels) # Or handle as appropriate for your data

    return {
        "text": row["input"], # Use 'input' key here
        "label": label_vector
    }

# Apply the function to create new dataframes
train_classification = train_df.apply(convert_to_classification, axis=1).to_list()
val_classification = val_df.apply(convert_to_classification, axis=1).to_list()

# Convert the list of dictionaries to a Dataset object
train_dataset = Dataset.from_list(train_classification)
val_dataset = Dataset.from_list(val_classification)


def tokenize(batch):
    # Tokenize the text and return the input_ids and attention_mask
    tokenized_inputs = tokenizer(batch["text"], padding="max_length", truncation=True)
    # Include the labels in the returned dictionary
    tokenized_inputs["label"] = batch["label"]
    return tokenized_inputs


# Tokenize the datasets - the 'label' column is already present from convert_to_classification
encoded_train = train_dataset.map(tokenize, batched=True)
encoded_val = val_dataset.map(tokenize, batched=True)

# Set the format of the datasets to 'torch'
encoded_train.set_format("torch", columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
encoded_val.set_format("torch", columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions # Get the raw logits

    # Convert labels to a 2D numpy array of floats
    true_labels_binary = np.array(labels).astype(float)

    # Apply sigmoid to logits to get probabilities
    sigmoid = lambda x: 1 / (1 + np.exp(-x))
    probabilities = sigmoid(preds)

    # Choose a threshold. A common starting point is 0.5
    threshold = 0.5
    predicted_labels_binary = (probabilities > threshold).astype(int)


    # Calculate metrics. For multi-label, 'weighted' F1 averages the F1 score
    # for each label, weighted by the number of true instances for each label.
    # Accuracy in multi-label can be interpreted in different ways (e.g., instance-based, label-based).
    # Here, we'll calculate label-based accuracy and weighted F1.

    # Label-based accuracy: proportion of instances where ALL labels were correctly predicted.
    label_accuracy = np.mean((predicted_labels_binary == true_labels_binary).all(axis=1))

    # Weighted F1-score:
    weighted_f1 = f1_score(true_labels_binary, predicted_labels_binary, average="weighted")

    # AUC-ROC score: Can be calculated per label and averaged, or micro/macro averaged.
    # Let's calculate the weighted AUC-ROC
    try:
        weighted_roc_auc = roc_auc_score(true_labels_binary, probabilities, average="weighted")
    except ValueError:
        # Handle cases where there's only one class present in a fold
        weighted_roc_auc = np.nan # Or some other indicator


    return {
        "label_accuracy": label_accuracy,
        "weighted_f1": weighted_f1,
        "weighted_roc_auc": weighted_roc_auc,
    }


training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4, # Reduced batch size
    per_device_eval_batch_size=4,  # Reduced batch size
    num_train_epochs=5, # Reduced number of epochs
    weight_decay=0.01,
    logging_dir='./logs',
    report_to="none",
    # Add parameters for multi-label classification if needed by the Trainer,
    # but generally, the loss function in AutoModelForSequenceClassification
    # handles multi-label when num_labels > 1 and a sigmoid is applied internally
    # before the loss calculation (e.g., BCEWithLogitsLoss).
    # Ensure your model output is logits before the sigmoid for the loss.
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train, # Labels are already in the dataset
    eval_dataset=encoded_val,   # Labels are already in the dataset
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/700 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/150 [00:00<?, ? examples/s]

/tmp/ipython-input-2011270898.py:167: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


RuntimeError: result type Float can't be cast to the desired output type Long

**Reasoning**:
The traceback indicates a `RuntimeError: result type Float can't be cast to the desired output type Long` during the calculation of the binary cross-entropy loss. This error happens because the target labels for `BCEWithLogitsLoss` are expected to be of type float, but they are currently of type long (or integer). I need to ensure that the 'label' column in the `encoded_train` and `encoded_val` datasets is of float type. I will modify the code to cast the labels to `torch.float` after tokenization and before setting the format.



In [29]:
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, jaccard_score
from datasets import Dataset # Import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import json
from google.colab import drive
import torch # Import torch
from transformers import DataCollatorWithPadding # Import DataCollatorWithPadding

# Mount Google Drive and define file path (assuming it's the same as before)
drive.mount('/content/drive')
file_path = "/content/drive/MyDrive/Datasets/LLMs/MIMIC/extracted_symptoms_new.jsonl"

# Load the data
data = []
with open(file_path, 'r') as f:
    for line in f:
        data.append(json.loads(line))

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)

# Split the DataFrame into training and temporary sets
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)

# Split the temporary set into validation and test sets
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)


# Define model_name and tokenizer
model_name = "emilyalsentzer/Bio_ClinicalBERT"  # or "dmis-lab/biobert-base-cased-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Collect all unique labels from train, val, and test sets
# Assuming 'output' contains a string of labels separated by commas and spaces
all_labels = set()
for dataset in [train_df['output'], val_df['output'], test_df['output']]:
    for labels_str in dataset:
        # Handle potential missing or non-string values
        if isinstance(labels_str, str):
            labels = [label.strip() for label in labels_str.split(',')]
            all_labels.update(labels)

unique_labels = list(all_labels)

# Create label to ID and ID to label mappings
label2id = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for label, i in label2id.items()}

# Load the pre-trained model
num_labels = len(unique_labels)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    problem_type="multi_label_classification" # Set problem type for multi-label
)

# Define a function to convert to classification format
def convert_to_classification(row):
    # Handle potential missing or non-string values in 'output'
    if isinstance(row["output"], str):
        labels = [label.strip() for label in row["output"].split(',')]
        # Create a multi-hot encoded label vector
        label_vector = [0] * len(unique_labels)
        for label in labels:
            if label in label2id:
                label_vector[label2id[label]] = 1
    else:
        # Handle cases where 'output' is not a string (e.g., NaN)
        label_vector = [0] * len(unique_labels) # Or handle as appropriate for your data

    return {
        "text": row["input"], # Use 'input' key here
        "label": label_vector
    }

# Apply the function to create new dataframes
train_classification = train_df.apply(convert_to_classification, axis=1).to_list()
val_classification = val_df.apply(convert_to_classification, axis=1).to_list()
test_classification = test_df.apply(convert_to_classification, axis=1).to_list() # Process test_df


# Convert the list of dictionaries to a Dataset object
train_dataset = Dataset.from_list(train_classification)
val_dataset = Dataset.from_list(val_classification)
test_dataset = Dataset.from_list(test_classification) # Create test_dataset


def tokenize_and_prepare_labels(batch):
    # Tokenize the text and return the input_ids and attention_mask
    tokenized_inputs = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=512)
    # Convert the list of labels to a float tensor
    tokenized_inputs["label"] = torch.tensor(batch["label"], dtype=torch.float)
    return tokenized_inputs


# Tokenize the datasets and prepare labels
encoded_train = train_dataset.map(tokenize_and_prepare_labels, batched=True)
encoded_val = val_dataset.map(tokenize_and_prepare_labels, batched=True)
encoded_test = test_dataset.map(tokenize_and_prepare_labels, batched=True) # Tokenize test_dataset


# Set the format of the datasets to 'torch'
encoded_train.set_format("torch", columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
encoded_val.set_format("torch", columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
encoded_test.set_format("torch", columns=['input_ids', 'token_type_ids', 'attention_mask', 'label']) # Set format for test_dataset


# Define a custom data collator
class MultiLabelDataCollatorWithPadding(DataCollatorWithPadding):
    def __call__(self, features):
        batch = super().__call__(features)
        # Convert labels to float tensor
        batch["labels"] = torch.tensor(batch["labels"], dtype=torch.float)
        return batch


# Instantiate the custom data collator
data_collator = MultiLabelDataCollatorWithPadding(tokenizer=tokenizer)


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions # Get the raw logits

    # Convert labels to a 2D numpy array of floats
    true_labels_binary = np.array(labels).astype(float)

    # Apply sigmoid to logits to get probabilities
    sigmoid = lambda x: 1 / (1 + np.exp(-x))
    probabilities = sigmoid(preds)

    # Choose a threshold. A common starting point is 0.5
    threshold = 0.5
    predicted_labels_binary = (probabilities > threshold).astype(int)


    # Calculate metrics. For multi-label, 'weighted' F1 averages the F1 score
    # for each label, weighted by the number of true instances for each label.
    # Accuracy in multi-label can be interpreted in different ways (e.g., instance-based, label-based).
    # Here, we'll calculate label-based accuracy and weighted F1.

    # Label-based accuracy: proportion of instances where ALL labels were correctly predicted.
    label_accuracy = np.mean((predicted_labels_binary == true_labels_binary).all(axis=1))

    # Weighted F1-score:
    weighted_f1 = f1_score(true_labels_binary, predicted_labels_binary, average="weighted")

    # AUC-ROC score: Can be calculated per label and averaged, or micro/macro averaged.
    # Let's calculate the weighted AUC-ROC
    try:
        weighted_roc_auc = roc_auc_score(true_labels_binary, probabilities, average="weighted")
    except ValueError:
        # Handle cases where there's only one class present in a fold
        weighted_roc_auc = np.nan # Or some other indicator

    # Jaccard Similarity
    # Calculate Jaccard score for each instance and then average
    jaccard = jaccard_score(true_labels_binary, predicted_labels_binary, average="samples")


    return {
        "label_accuracy": label_accuracy,
        "weighted_f1": weighted_f1,
        "weighted_roc_auc": weighted_roc_auc,
        "jaccard_similarity": jaccard # Add jaccard similarity
    }


training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",  # Set logging strategy to epoch
    learning_rate=2e-5,
    per_device_train_batch_size=4, # Reduced batch size
    per_device_eval_batch_size=4,  # Reduced batch size
    num_train_epochs=5, # Reduced number of epochs
    weight_decay=0.01,
    logging_dir='./logs',
    report_to="none",
    # Add parameters for multi-label classification if needed by the Trainer,
    # but generally, the loss function in AutoModelForSequenceClassification
    # handles multi-label when num_labels > 1 and a sigmoid is applied internally
    # before the loss calculation (e.g., BCEWithLogitsLoss).
    # Ensure your model output is logits before the sigmoid for the loss.
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train, # Labels are already in the dataset
    eval_dataset=encoded_val,   # Labels are already in the dataset
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator # Pass the custom data collator
)

trainer.train()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/700 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

/tmp/ipython-input-2001163551.py:193: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/tmp/ipython-input-2001163551.py:118: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.float)


Epoch,Training Loss,Validation Loss,Label Accuracy,Weighted F1,Weighted Roc Auc,Jaccard Similarity
1,0.350800,0.144061,0.000000,0.037410,0.494417,0.050143
2,0.104800,0.078004,0.000000,0.037410,0.492656,0.050143
3,0.070100,0.061373,0.000000,0.037410,0.492490,0.050143
4,0.059500,0.055313,0.000000,0.037410,0.492057,0.050143
5,0.055700,0.053628,0.000000,0.037410,0.493481,0.050143


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarnin

TrainOutput(global_step=875, training_loss=0.12817192840576172, metrics={'train_runtime': 176.5205, 'train_samples_per_second': 19.828, 'train_steps_per_second': 4.957, 'total_flos': 933869905920000.0, 'train_loss': 0.12817192840576172, 'epoch': 5.0})

In [32]:
print(unique_labels)

['Bone marrow replaced by transplant', 'without mention of open intracranial wound', 'Paralysis agitans', 'Hemoperitoneum (nontraumatic)', 'type II or unspecified type', 'Closed fracture of three ribs', 'Attention deficit disorder with hyperactivity', 'Contusion of thigh', 'Secondary malignant neoplasm of brain and spinal cord', 'Acute on chronic diastolic heart failure', 'Closed fracture of seventh cervical vertebra', 'without neurogenic claudication', 'Other specified disorders of arteries and arterioles', 'Chronic diastolic heart failure', 'buttock', 'Secondary hypercoagulable state', 'without mention of obstruction or gangrene', 'Intestinal disaccharidase deficiencies and disaccharide malabsorption', 'Closed supracondylar fracture of femur', 'Accidental poisoning by anticonvulsant and anti-parkinsonism drugs', 'Alkalosis', 'Other fall', 'Gout', 'Mechanical complication due to cardiac pacemaker (electrode)', 'thoracic region', 'Pyogenic arthritis', 'Adrenal cortical steroids causing

In [33]:
len(unique_labels)

1572

In [35]:
# Make predictions on a subset of the test dataset
predictions = trainer.predict(encoded_test.select(range(5))) # Predict on the first 5 examples

# Get the predicted logits
predicted_logits = predictions.predictions

# Apply sigmoid to get probabilities
sigmoid = lambda x: 1 / (1 + np.exp(-x))
predicted_probabilities = sigmoid(predicted_logits)

# Choose a threshold (e.g., 0.5) to get binary predictions
threshold = 0.5
predicted_labels_binary = (predicted_probabilities > threshold).astype(int)

# Map predicted binary labels back to original labels
predicted_labels_text = []
for prediction in predicted_labels_binary:
    predicted_labels = [id2label[i] for i, label in enumerate(prediction) if label == 1]
    predicted_labels_text.append(predicted_labels)

# Print the predicted labels as text
print("Predicted Labels (text) for the first 5 test examples:")
for i, labels in enumerate(predicted_labels_text):
    print(f"Example {i+1}: {labels}")

/tmp/ipython-input-2001163551.py:118: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.float)


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarnin

Predicted Labels (text) for the first 5 test examples:
Example 1: ['unspecified']
Example 2: ['unspecified']
Example 3: ['unspecified']
Example 4: ['unspecified']
Example 5: ['unspecified']


In [31]:
# Evaluate the model on the test dataset
test_results = trainer.evaluate(encoded_test)

print("Test Results:")
print(test_results)

/tmp/ipython-input-2001163551.py:118: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.float)


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarnin

Test Results:
{'eval_loss': 0.05548809841275215, 'eval_label_accuracy': 0.0, 'eval_weighted_f1': 0.03611444577831133, 'eval_weighted_roc_auc': 0.48523125374179127, 'eval_jaccard_similarity': 0.04933348393120827, 'eval_runtime': 2.6993, 'eval_samples_per_second': 55.57, 'eval_steps_per_second': 14.078, 'epoch': 5.0}


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.12/dist

**Reasoning**:
The same `RuntimeError` persists even after explicitly casting the 'label' column to `torch.float`. This suggests that the casting might not be applied correctly or the issue lies elsewhere in the data preparation pipeline before the loss calculation. To ensure the labels are correctly formatted as floats, I will modify the `tokenize` function to cast the labels to `torch.float` before returning the tokenized inputs. This should guarantee the correct data type when the labels are accessed during training.



In [12]:
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from datasets import Dataset # Import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import json
from google.colab import drive
import torch # Import torch

# Mount Google Drive and define file path (assuming it's the same as before)
drive.mount('/content/drive')
file_path = "/content/drive/MyDrive/Datasets/LLMs/MIMIC/extracted_symptoms_new.jsonl"

# Load the data
data = []
with open(file_path, 'r') as f:
    for line in f:
        data.append(json.loads(line))

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)

# Split the DataFrame into training and temporary sets
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)

# Split the temporary set into validation and test sets
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)


# Define model_name and tokenizer
model_name = "emilyalsentzer/Bio_ClinicalBERT"  # or "dmis-lab/biobert-base-cased-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Collect all unique labels from train, val, and test sets
# Assuming 'output' contains a string of labels separated by commas and spaces
all_labels = set()
for dataset in [train_df['output'], val_df['output'], test_df['output']]:
    for labels_str in dataset:
        # Handle potential missing or non-string values
        if isinstance(labels_str, str):
            labels = [label.strip() for label in labels_str.split(',')]
            all_labels.update(labels)

unique_labels = list(all_labels)

# Create label to ID and ID to label mappings
label2id = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for label, i in label2id.items()}

# Load the pre-trained model
num_labels = len(unique_labels)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    problem_type="multi_label_classification" # Set problem type for multi-label
)

# Define a function to convert to classification format
def convert_to_classification(row):
    # Handle potential missing or non-string values in 'output'
    if isinstance(row["output"], str):
        labels = [label.strip() for label in row["output"].split(',')]
        # Create a multi-hot encoded label vector
        label_vector = [0] * len(unique_labels)
        for label in labels:
            if label in label2id:
                label_vector[label2id[label]] = 1
    else:
        # Handle cases where 'output' is not a string (e.g., NaN)
        label_vector = [0] * len(unique_labels) # Or handle as appropriate for your data

    return {
        "text": row["input"], # Use 'input' key here
        "label": label_vector
    }

# Apply the function to create new dataframes
train_classification = train_df.apply(convert_to_classification, axis=1).to_list()
val_classification = val_df.apply(convert_to_classification, axis=1).to_list()

# Convert the list of dictionaries to a Dataset object
train_dataset = Dataset.from_list(train_classification)
val_dataset = Dataset.from_list(val_classification)


def tokenize(batch):
    # Tokenize the text and return the input_ids and attention_mask
    tokenized_inputs = tokenizer(batch["text"], padding="max_length", truncation=True)
    # Include the labels in the returned dictionary and cast to float
    tokenized_inputs["label"] = [torch.tensor(label, dtype=torch.float) for label in batch["label"]]
    return tokenized_inputs


# Tokenize the datasets - the 'label' column is already present from convert_to_classification
encoded_train = train_dataset.map(tokenize, batched=True)
encoded_val = val_dataset.map(tokenize, batched=True)

# Set the format of the datasets to 'torch'
encoded_train.set_format("torch", columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
encoded_val.set_format("torch", columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions # Get the raw logits

    # Convert labels to a 2D numpy array of floats
    true_labels_binary = np.array(labels).astype(float)

    # Apply sigmoid to logits to get probabilities
    sigmoid = lambda x: 1 / (1 + np.exp(-x))
    probabilities = sigmoid(preds)

    # Choose a threshold. A common starting point is 0.5
    threshold = 0.5
    predicted_labels_binary = (probabilities > threshold).astype(int)


    # Calculate metrics. For multi-label, 'weighted' F1 averages the F1 score
    # for each label, weighted by the number of true instances for each label.
    # Accuracy in multi-label can be interpreted in different ways (e.g., instance-based, label-based).
    # Here, we'll calculate label-based accuracy and weighted F1.

    # Label-based accuracy: proportion of instances where ALL labels were correctly predicted.
    label_accuracy = np.mean((predicted_labels_binary == true_labels_binary).all(axis=1))

    # Weighted F1-score:
    weighted_f1 = f1_score(true_labels_binary, predicted_labels_binary, average="weighted")

    # AUC-ROC score: Can be calculated per label and averaged, or micro/macro averaged.
    # Let's calculate the weighted AUC-ROC
    try:
        weighted_roc_auc = roc_auc_score(true_labels_binary, probabilities, average="weighted")
    except ValueError:
        # Handle cases where there's only one class present in a fold
        weighted_roc_auc = np.nan # Or some other indicator


    return {
        "label_accuracy": label_accuracy,
        "weighted_f1": weighted_f1,
        "weighted_roc_auc": weighted_roc_auc,
    }


training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4, # Reduced batch size
    per_device_eval_batch_size=4,  # Reduced batch size
    num_train_epochs=5, # Reduced number of epochs
    weight_decay=0.01,
    logging_dir='./logs',
    report_to="none",
    # Add parameters for multi-label classification if needed by the Trainer,
    # but generally, the loss function in AutoModelForSequenceClassification
    # handles multi-label when num_labels > 1 and a sigmoid is applied internally
    # before the loss calculation (e.g., BCEWithLogitsLoss).
    # Ensure your model output is logits before the sigmoid for the loss.
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train, # Labels are already in the dataset
    eval_dataset=encoded_val,   # Labels are already in the dataset
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/700 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/150 [00:00<?, ? examples/s]

/tmp/ipython-input-126969128.py:167: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


RuntimeError: result type Float can't be cast to the desired output type Long

**Reasoning**:
The `RuntimeError` persists, indicating the labels are still not in the expected float format when passed to the loss function. The previous attempt to cast within the `tokenize` function might not be effective with `batched=True`. A more reliable approach is to ensure the labels are float tensors when the dataset is created from the list of dictionaries. I will modify the `convert_to_classification` function to return the label vector as a torch float tensor and then create the datasets from these lists of dictionaries.



In [13]:
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from datasets import Dataset # Import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import json
from google.colab import drive
import torch # Import torch

# Mount Google Drive and define file path (assuming it's the same as before)
drive.mount('/content/drive')
file_path = "/content/drive/MyDrive/Datasets/LLMs/MIMIC/extracted_symptoms_new.jsonl"

# Load the data
data = []
with open(file_path, 'r') as f:
    for line in f:
        data.append(json.loads(line))

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)

# Split the DataFrame into training and temporary sets
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)

# Split the temporary set into validation and test sets
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)


# Define model_name and tokenizer
model_name = "emilyalsentzer/Bio_ClinicalBERT"  # or "dmis-lab/biobert-base-cased-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Collect all unique labels from train, val, and test sets
# Assuming 'output' contains a string of labels separated by commas and spaces
all_labels = set()
for dataset in [train_df['output'], val_df['output'], test_df['output']]:
    for labels_str in dataset:
        # Handle potential missing or non-string values
        if isinstance(labels_str, str):
            labels = [label.strip() for label in labels_str.split(',')]
            all_labels.update(labels)

unique_labels = list(all_labels)

# Create label to ID and ID to label mappings
label2id = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for label, i in label2id.items()}

# Load the pre-trained model
num_labels = len(unique_labels)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    problem_type="multi_label_classification" # Set problem type for multi-label
)

# Define a function to convert to classification format
def convert_to_classification(row):
    # Handle potential missing or non-string values in 'output'
    if isinstance(row["output"], str):
        labels = [label.strip() for label in row["output"].split(',')]
        # Create a multi-hot encoded label vector
        label_vector = [0] * len(unique_labels)
        for label in labels:
            if label in label2id:
                label_vector[label2id[label]] = 1
    else:
        # Handle cases where 'output' is not a string (e.g., NaN)
        label_vector = [0] * len(unique_labels) # Or handle as appropriate for your data

    return {
        "text": row["input"], # Use 'input' key here
        "label": torch.tensor(label_vector, dtype=torch.float) # Convert label vector to float tensor here
    }

# Apply the function to create new dataframes
train_classification = train_df.apply(convert_to_classification, axis=1).to_list()
val_classification = val_df.apply(convert_to_classification, axis=1).to_list()

# Convert the list of dictionaries to a Dataset object
train_dataset = Dataset.from_list(train_classification)
val_dataset = Dataset.from_list(val_classification)


def tokenize(batch):
    # Tokenize the text and return the input_ids and attention_mask
    tokenized_inputs = tokenizer(batch["text"], padding="max_length", truncation=True)
    # Labels are already in the correct format (float tensor) from convert_to_classification
    tokenized_inputs["label"] = batch["label"]
    return tokenized_inputs


# Tokenize the datasets - the 'label' column is already present from convert_to_classification
encoded_train = train_dataset.map(tokenize, batched=True)
encoded_val = val_dataset.map(tokenize, batched=True)

# Set the format of the datasets to 'torch'
encoded_train.set_format("torch", columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
encoded_val.set_format("torch", columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions # Get the raw logits

    # Convert labels to a 2D numpy array of floats
    true_labels_binary = np.array(labels).astype(float)

    # Apply sigmoid to logits to get probabilities
    sigmoid = lambda x: 1 / (1 + np.exp(-x))
    probabilities = sigmoid(preds)

    # Choose a threshold. A common starting point is 0.5
    threshold = 0.5
    predicted_labels_binary = (probabilities > threshold).astype(int)


    # Calculate metrics. For multi-label, 'weighted' F1 averages the F1 score
    # for each label, weighted by the number of true instances for each label.
    # Accuracy in multi-label can be interpreted in different ways (e.g., instance-based, label-based).
    # Here, we'll calculate label-based accuracy and weighted F1.

    # Label-based accuracy: proportion of instances where ALL labels were correctly predicted.
    label_accuracy = np.mean((predicted_labels_binary == true_labels_binary).all(axis=1))

    # Weighted F1-score:
    weighted_f1 = f1_score(true_labels_binary, predicted_labels_binary, average="weighted")

    # AUC-ROC score: Can be calculated per label and averaged, or micro/macro averaged.
    # Let's calculate the weighted AUC-ROC
    try:
        weighted_roc_auc = roc_auc_score(true_labels_binary, probabilities, average="weighted")
    except ValueError:
        # Handle cases where there's only one class present in a fold
        weighted_roc_auc = np.nan # Or some other indicator


    return {
        "label_accuracy": label_accuracy,
        "weighted_f1": weighted_f1,
        "weighted_roc_auc": weighted_roc_auc,
    }


training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4, # Reduced batch size
    per_device_eval_batch_size=4,  # Reduced batch size
    num_train_epochs=5, # Reduced number of epochs
    weight_decay=0.01,
    logging_dir='./logs',
    report_to="none",
    # Add parameters for multi-label classification if needed by the Trainer,
    # but generally, the loss function in AutoModelForSequenceClassification
    # handles multi-label when num_labels > 1 and a sigmoid is applied internally
    # before the loss calculation (e.g., BCEWithLogitsLoss).
    # Ensure your model output is logits before the sigmoid for the loss.
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train, # Labels are already in the dataset
    eval_dataset=encoded_val,   # Labels are already in the dataset
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/700 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


KeyboardInterrupt: 